In [3]:
import wikipedia
def answerfromwiki(text): 
    complete_content = wikipedia.summary(text)
    print(complete_content)
    return complete_content

In [ ]:
from flask import Flask, render_template, request, session, url_for, redirect, jsonify
import pymysql

#=========
import nltk
from nltk.stem.lancaster import LancasterStemmer

stemmer = LancasterStemmer()
import numpy
import tflearn
import tensorflow
import json
import pickle

from sklearn import preprocessing
import pandas as pd
import numpy as np
import scipy.stats
import scipy.spatial
from sklearn.model_selection import KFold
import random
from sklearn.metrics import mean_squared_error
from math import sqrt
import math
import warnings
import sys
#from sklearn.utils.extmath import np.dot

#=========Database Connection===
connection = pymysql.connect(host="localhost", user="root", password="", database="063autoqa")
cursor = connection.cursor()
#============start=======chatbot============
#chatbot
with open("QASystemCrypto.json", encoding="utf8") as file:
    data = json.load(file)

try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            # print(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model11 = tflearn.DNN(net)

try:
    model11.load("model.tflearn")
except:

    model11.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model11.save("model.tflearn")


#==============end====chatbot=================


app = Flask(__name__)
app.secret_key = 'random string'


@app.route('/index')
@app.route('/')
def index():
    return render_template('index.html')


@app.route('/register', methods=["GET","POST"])
def register():
    if request.method == "POST":
        name = request.form.get("name")
        email = request.form.get("email")
        mobile = request.form.get("mobile")
        gender = request.form.get("gender")
        dob = request.form.get("dob")
        username = request.form.get("username")
        password = request.form.get("password")
        cursor.execute("insert into userdetails(fullname,gender,mobile,email,dob,username,password) values('"+name+"','"+gender+"','"+mobile+"','"+email+"','"+dob+"','"+username+"','"+password+"')")
        connection.commit()
        #return render_template('/index')
        return render_template('index.html')
    else:
        #return render_template('index.html')
        return render_template('index.html')


@app.route('/login', methods=["GET","POST"])
def login():
    msg = ''
    if request.method == "POST":
        session.pop('user',None)
        username = request.form.get("username")
        password = request.form.get("password")
        cursor.execute('SELECT * FROM userdetails WHERE username = %s AND password = %s', (username, password))
        account = cursor.fetchone()
        #print(account)
        if account:
            session['user'] = account[1]
            #return render_template('home.html')
            return redirect(url_for('chatbot'))
        else:
            # Account doesnt exist or username/password incorrect
            msg = 'Incorrect username/password!'
    #return render_template('index.html', msg=msg)
    return redirect(url_for('index'))


#logout code
@app.route('/logout')
def logout():
    session.pop('user')
    return redirect(url_for('index'))


@app.route('/home')
def home():
    if 'user' in session:
        return render_template('home.html', user=session['user'])
    return redirect(url_for('index'))

@app.route('/aboutus')
def aboutus():
    return render_template('aboutus.html')


@app.route('/contactus')
def contactus():
    return render_template('contactus.html')
#bot start======

@app.route('/chatbot')
def chatbot():
    if 'user' in session:
        return render_template('chatbot.html', user=session['user'])
    return redirect(url_for('index'))


@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    results = model11.predict([bag_of_words(userText, words)])
    print(results)
    results_index = numpy.argmax(results)
    print(results_index)
    print(results[0][results_index])
    oppred=results[0][results_index]
    outdatagot = "No data found for this query in our dataset \n"
    if oppred>0.7:
        tag = labels[results_index]
        #outdatagot = ""
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
                outdatagot = random.choice(responses)
                print(outdatagot)
        return str(outdatagot)
    else:
        outdatagot=outdatagot+" Answer from wikipedia \n"+answerfromwiki(userText)
        return str(outdatagot)


def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    return numpy.array(bag)
model11.load("model.tflearn")
#bot end======-------------


if __name__ == '__main__':
    #app.run(debug="True")
    app.run('0.0.0.0')


C:\Users\Ningesh\AppData\Roaming\Python\Python37\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Ningesh\AppData\Roaming\Python\Python37\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Ningesh\AppData\Roaming\Python\Python37\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Ningesh\AppData\

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
curses is not supported on this machine (please install/reinstall curses for an optimal experience)










INFO:tensorflow:Restoring parameters from C:\Users\Ningesh\063AutoQA\model.tflearn
INFO:tensorflow:Restoring parameters from C:\Users\Ningesh\063AutoQA\model.tflearn
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Jun/2020 11:33:44] "GET /static/digital.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [23/Jun/2020 11:33:44] "GET /static/digital.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [23/Jun/2020 11:33:45] "GET /static/digital.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [23/Jun/2020 11:33:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2020 11:33:47] "GET /static/images/banner.png HTTP/1.1" 404 -
127.0.0.1 - - [23/Jun/2020 11:33:53] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [23/Jun/2020 11:33:53] "GET /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2020 11:33:54] "GET /static/digital.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [23/Jun/2020 11:33:55] "GET /static/digital.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [23/Jun/2020 11:34:27] "GET /get?msg=nlp HTTP/1.1" 200 -


[[4.35677327e-25 4.82713767e-21 4.18832297e-15 7.27169269e-31
  6.23853134e-24 7.16293802e-10 4.48972990e-23 7.61878326e-30
  7.82194291e-25 1.78971524e-25 9.35926938e-28 5.85414220e-25
  7.91802635e-36 9.54509477e-22 3.22457007e-14 3.57087703e-16
  1.59873756e-32 4.06398904e-36 1.66081281e-11 2.27508690e-06
  1.88370261e-12 2.70327574e-16 6.35801088e-16 1.53898164e-24
  1.76671872e-19 2.59847937e-28 3.89788858e-23 1.61730319e-12
  2.11121211e-20 6.41645986e-17 1.26033118e-34 2.91965206e-30
  1.59042507e-21 1.68020351e-19 7.62909791e-10 5.51623134e-15
  1.37015160e-24 2.23123493e-07 1.87126378e-29 7.40857842e-32
  4.41759123e-32 1.68671053e-32 3.27190809e-11 2.50775233e-24
  5.80504691e-23 1.13738344e-23 5.24850413e-16 3.02897748e-33
  5.14671100e-32 1.15111076e-16 5.59103293e-31 1.45994611e-21
  1.44029652e-15 2.87273252e-07 1.08167244e-26 6.24564952e-32
  7.19293005e-30 6.72095019e-11 4.60437403e-25 6.72542910e-10
  3.38718973e-16 1.47921599e-14 4.54922731e-11 1.35638176e-08
  4.7410

127.0.0.1 - - [23/Jun/2020 11:34:53] "GET /get?msg=DigitalImage%20Forensics HTTP/1.1" 200 -


...

127.0.0.1 - - [23/Jun/2020 11:36:28] "GET /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2020 11:36:35] "GET /static/digital.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [23/Jun/2020 11:36:35] "GET /static/digital.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [23/Jun/2020 11:36:35] "GET /static/digital.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [23/Jun/2020 11:36:36] "GET /static/digital.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [23/Jun/2020 11:36:36] "GET /static/digital.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [23/Jun/2020 11:36:36] "GET /static/digital.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [23/Jun/2020 11:36:45] "GET /get?msg=what%20is%20Digital%20Image%20Forensics HTTP/1.1" 200 -


[[2.0150344e-08 1.9509662e-25 0.0000000e+00 1.8199347e-20 6.3481734e-13
  0.0000000e+00 3.5459199e-33 9.9998724e-01 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 6.8616382e-33 0.0000000e+00 0.0000000e+00
  6.4633734e-32 1.1316261e-08 7.0904572e-16 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 1.9875436e-23 1.9068746e-28 0.0000000e+00
  1.0186317e-18 0.0000000e+00 0.0000000e+00 2.9897876e-07 0.0000000e+00
  0.0000000e+00 0.0000000e+00 2.2407069e-31 0.0000000e+00 0.0000000e+00
  3.5038033e-20 4.0691947e-07 0.0000000e+00 0.0000000e+00 2.7173141e-30
  1.7329365e-25 1.0255822e-06 0.0000000e+00 6.2946461e-11 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  2.1709392e-31 0.0000000e+00 2.1088935e-18 0.0000000e+00 3.8117668e-20
  0.0000000e+00 2.3628834e-27 0.0000000e+00 0.0000000e+00 0.0000000e+00
  2.8883421e-38 8.9820076e-30 2.0422044e-30 2.1269900e-34 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.8730

127.0.0.1 - - [23/Jun/2020 11:37:17] "GET /get?msg=what%20is%20Digital%20Video%2FAudio%20Forensics HTTP/1.1" 200 -


[[8.6197793e-19 7.3238814e-21 0.0000000e+00 8.0250467e-28 2.2625149e-21
  0.0000000e+00 3.7135005e-22 8.2421252e-07 4.9918422e-29 0.0000000e+00
  1.0911174e-28 0.0000000e+00 3.7231642e-20 7.2907030e-23 0.0000000e+00
  3.1298093e-25 1.4823623e-13 5.1255228e-08 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 2.5676468e-09 0.0000000e+00
  9.7883377e-12 1.2924377e-26 0.0000000e+00 1.8216349e-19 0.0000000e+00
  4.4973844e-29 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  1.2109811e-29 2.5654855e-22 0.0000000e+00 2.3006048e-32 1.5566251e-10
  1.7131077e-06 9.9999738e-01 0.0000000e+00 1.6060867e-10 0.0000000e+00
  0.0000000e+00 7.4740907e-28 7.0851483e-31 1.2688998e-35 0.0000000e+00
  8.3363350e-34 0.0000000e+00 5.7201387e-30 0.0000000e+00 3.2576616e-28
  0.0000000e+00 1.4638774e-24 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 2.2583682e-35 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 1.5858202e-32 0.0000000e+00 1.9839

127.0.0.1 - - [23/Jun/2020 11:37:43] "GET /static/digital.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [23/Jun/2020 11:37:43] "GET /static/digital.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [23/Jun/2020 11:37:44] "GET /static/digital.mp4 HTTP/1.1" 206 -
